In [4]:
import sys
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import resnet50
from keras_applications import resnext
import pandas as pd
import numpy as np
import os
import cv2

In [2]:
labels = pd.read_csv('Data/train_label.csv')
labels['Type'].value_counts()
sample = pd.read_csv('Data/sample_submission.csv')
num_train_images = 2520  # we choose 3300 images for this assignment. It works for a machine having 8Gb Ram. You can adjust it if your Ram is different. 
split_point = 2240 # split the data into training data [0:3000] and val data [3000:]
print('num_train_images:', num_train_images)
types = sample.columns[1:]
print('Types:', types)
num_classes = len(types)
print('num_classes:', num_classes)

num_train_images: 2520
Types: Index(['distribute', 'ineq', 'integral', 'limit', 'matrix', 'series', 'sqrt'], dtype='object')
num_classes: 7


In [5]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

img_width = 224

def get_image(filename):
    ########################################################################
    # TODO: Your code here...
    ########################################################################
    original = load_img(filename, target_size=(224,224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    return image_batch[0]

In [6]:
x_train = np.zeros((num_train_images, img_width, img_width, 3), dtype=np.uint8)
y_train = np.zeros((num_train_images, num_classes), dtype=np.uint8)

count = 0

for i in range(num_train_images):
    x_train[i] = get_image('Data/train/%s.png' % labels['id'][i])
    pos_arrays = (types == labels['Type'][i]).nonzero() # recall that types is the array of classes
    pos = pos_arrays[0][0]
    y_train[i][pos] = 1
    count += 1
    if(count % 1000 == 0): print(count, 'images loaded')

1000 images loaded
2000 images loaded


### Xception with nothing changed

In [7]:
from tensorflow.keras.applications import xception
xception_x_train = xception.preprocess_input(x_train.copy())
xception_model = xception.Xception(weights='imagenet', include_top=False)
xception_features = xception_model.predict(xception_x_train, batch_size=64, verbose=1)
print(xception_features.shape)

40/40 [==============================] - 187s 5s/step
(2520, 7, 7, 2048)


In [8]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Activation, BatchNormalization

inputs = Input(shape = (7, 7, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [9]:
from tensorflow.keras.models import Model

model_x = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_x.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 7, 7, 2048)]      0         
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 500)               1024500   
                                                                 
 batch_normalization_4 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 500)               0     

In [10]:
model_x.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [11]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_x.fit(xception_features[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(xception_features[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)


Epoch 1/30
18/18 [==============================] - 6s 52ms/step - loss: 1.4119 - accuracy: 0.5353 - val_loss: 1.9369 - val_accuracy: 0.0393
Epoch 2/30
18/18 [==============================] - 1s 35ms/step - loss: 0.8633 - accuracy: 0.6893 - val_loss: 1.2222 - val_accuracy: 0.6857
Epoch 3/30
18/18 [==============================] - 1s 38ms/step - loss: 0.7101 - accuracy: 0.7478 - val_loss: 0.9915 - val_accuracy: 0.8214
Epoch 4/30
18/18 [==============================] - 1s 36ms/step - loss: 0.6323 - accuracy: 0.7759 - val_loss: 1.0236 - val_accuracy: 0.7107
Epoch 5/30
18/18 [==============================] - 1s 36ms/step - loss: 0.5736 - accuracy: 0.8054 - val_loss: 0.8390 - val_accuracy: 0.7821
Epoch 6/30
18/18 [==============================] - 1s 36ms/step - loss: 0.5225 - accuracy: 0.8152 - val_loss: 0.8310 - val_accuracy: 0.7393
Epoch 7/30
18/18 [==============================] - 1s 36ms/step - loss: 0.5064 - accuracy: 0.8254 - val_loss: 0.7009 - val_accuracy: 0.8179
Epoch 8/30
18

In [38]:
model_x.save('model_x.h5') 

In [39]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = xception.preprocess_input(image_batch)
        feature_input = xception_model.predict(image_batch)
        predictions = model_x.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)


In [40]:
y_test = pd.read_excel(r'./Data/test_2_label.xlsx',usecols=[1,1])
y_test = y_test.to_numpy()
y_test = np.squeeze(y_test)

In [41]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.2857142857142857


# Xception with Activation = Sigmoid